# Deep Socket.IO Streaming with LangGraph

LangGraph의 **다중 노드 그래프**를 구성하고, 각 단계별 이벤트를 Socket.IO로 스트리밍합니다.

## 그래프 구조

```
START → [analyzer] → [generator] → [saver] → END
           ↓              ↓            ↓
        LLM 호출       LLM 호출     DB 저장
```

## 이벤트 타입

| 이벤트 | 설명 |
|--------|------|
| `node_start` | 노드 시작 (노드 이름 포함) |
| `node_end` | 노드 완료 (노드 이름 포함) |
| `token` | LLM 토큰 스트리밍 |
| `db_save` | DB 저장 완료 |
| `done` | 전체 완료 |

## 1. 환경 설정

In [ ]:
import os
import asyncio
import uuid
from datetime import datetime
from typing import Annotated, TypedDict, List
from dotenv import load_dotenv

load_dotenv()

OLLAMA_URL = os.getenv("OLLAMA_URL", "http://localhost:11434")
MODEL_NAME = "gemma3:12b"

print(f"Ollama URL: {OLLAMA_URL}")
print(f"Model: {MODEL_NAME}")

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

llm = ChatOllama(
    model=MODEL_NAME,
    base_url=OLLAMA_URL,
    temperature=0.7,
)

print(f"{MODEL_NAME} 모델 초기화 완료!")

## 2. Mocking DB 정의

In [ ]:
# Mocking Database
class MockDatabase:
    """간단한 인메모리 데이터베이스 시뮬레이션"""
    
    def __init__(self):
        self.records: List[dict] = []
    
    async def save(self, data: dict) -> dict:
        """데이터 저장 (비동기 시뮬레이션)"""
        await asyncio.sleep(0.5)  # DB 저장 지연 시뮬레이션
        
        record = {
            "id": str(uuid.uuid4()),
            "created_at": datetime.now().isoformat(),
            **data
        }
        self.records.append(record)
        return record
    
    def get_all(self) -> List[dict]:
        return self.records


# 전역 DB 인스턴스
mock_db = MockDatabase()
print("MockDatabase 초기화 완료!")

## 3. 그래프 상태 정의

In [ ]:
class GraphState(TypedDict):
    """그래프 상태"""
    # 원본 입력
    user_input: str
    
    # 분석 결과
    analysis: str
    
    # 생성된 응답
    response: str
    
    # DB 저장 결과
    saved_record: dict


print("GraphState 정의 완료!")

## 4. 노드 함수 정의

In [ ]:
async def analyzer_node(state: GraphState) -> GraphState:
    """
    [Node 1: Analyzer]
    사용자 입력을 분석하여 의도와 키워드를 추출합니다.
    """
    user_input = state["user_input"]
    
    prompt = f"""다음 사용자 입력을 분석해주세요.
- 주요 의도
- 핵심 키워드
- 감정 톤

사용자 입력: {user_input}

분석 결과를 간단히 작성해주세요."""
    
    response = await llm.ainvoke(prompt)
    
    return {"analysis": response.content}


async def generator_node(state: GraphState) -> GraphState:
    """
    [Node 2: Generator]
    분석 결과를 바탕으로 최종 응답을 생성합니다.
    """
    user_input = state["user_input"]
    analysis = state["analysis"]
    
    prompt = f"""다음 분석을 바탕으로 사용자에게 적절한 응답을 생성해주세요.

사용자 원본 입력: {user_input}

분석 결과:
{analysis}

위 분석을 고려하여 친절하고 도움이 되는 응답을 작성해주세요."""
    
    response = await llm.ainvoke(prompt)
    
    return {"response": response.content}


async def saver_node(state: GraphState) -> GraphState:
    """
    [Node 3: Saver]
    대화 내용을 DB에 저장합니다.
    """
    record = await mock_db.save({
        "user_input": state["user_input"],
        "analysis": state["analysis"],
        "response": state["response"],
    })
    
    return {"saved_record": record}


print("노드 함수 정의 완료!")

## 5. 그래프 구성

In [ ]:
# 그래프 생성
workflow = StateGraph(GraphState)

# 노드 추가
workflow.add_node("analyzer", analyzer_node)
workflow.add_node("generator", generator_node)
workflow.add_node("saver", saver_node)

# 엣지 정의 (순차 실행)
workflow.add_edge(START, "analyzer")
workflow.add_edge("analyzer", "generator")
workflow.add_edge("generator", "saver")
workflow.add_edge("saver", END)

# 컴파일
graph = workflow.compile()

print("그래프 컴파일 완료!")

In [ ]:
# 그래프 시각화
try:
    from IPython.display import Image, display
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"시각화 실패: {e}")
    print("\n그래프 구조:")
    print("START → analyzer → generator → saver → END")

## 6. 기본 실행 테스트

In [ ]:
# 기본 실행 (스트리밍 없이)
result = await graph.ainvoke({"user_input": "Python으로 웹 크롤러를 만들고 싶어요"})

print("[결과]")
print(f"\n📊 분석:\n{result['analysis'][:200]}...")
print(f"\n💬 응답:\n{result['response'][:200]}...")
print(f"\n💾 저장된 레코드:\n{result['saved_record']}")

## 7. 이벤트 기반 스트리밍 테스트

각 노드의 시작/끝과 LLM 토큰을 이벤트로 받습니다.

In [ ]:
async def stream_with_events(user_input: str):
    """
    이벤트 기반 스트리밍 - Socket.IO emit 시뮬레이션
    """
    print(f"{'='*60}")
    print(f"입력: {user_input}")
    print(f"{'='*60}\n")
    
    current_node = None
    
    async for event in graph.astream_events(
        {"user_input": user_input},
        version="v2"
    ):
        kind = event["event"]
        
        # 노드 시작
        if kind == "on_chain_start":
            node_name = event.get("name", "")
            if node_name in ["analyzer", "generator", "saver"]:
                current_node = node_name
                print(f"\n🚀 [NODE_START] {node_name}")
                # Socket.IO: await sio.emit("node_start", {"node": node_name}, to=sid)
        
        # 노드 종료
        elif kind == "on_chain_end":
            node_name = event.get("name", "")
            if node_name in ["analyzer", "generator", "saver"]:
                print(f"\n✅ [NODE_END] {node_name}")
                # Socket.IO: await sio.emit("node_end", {"node": node_name}, to=sid)
                
                # saver 노드 완료 시 DB 저장 이벤트
                if node_name == "saver":
                    output = event.get("data", {}).get("output", {})
                    if "saved_record" in output:
                        print(f"\n💾 [DB_SAVE] {output['saved_record']}")
                        # Socket.IO: await sio.emit("db_save", output["saved_record"], to=sid)
        
        # LLM 토큰 스트리밍
        elif kind == "on_chat_model_stream":
            chunk = event.get("data", {}).get("chunk")
            if chunk and chunk.content:
                print(chunk.content, end="", flush=True)
                # Socket.IO: await sio.emit("token", {"node": current_node, "content": chunk.content}, to=sid)
    
    print(f"\n\n{'='*60}")
    print("🎉 [DONE] 전체 완료")
    # Socket.IO: await sio.emit("done", to=sid)

In [ ]:
# 이벤트 스트리밍 테스트
await stream_with_events("FastAPI로 REST API 서버를 만드는 방법을 알려주세요")

## 8. 저장된 레코드 확인

In [ ]:
print("[저장된 모든 레코드]\n")
for i, record in enumerate(mock_db.get_all(), 1):
    print(f"--- Record {i} ---")
    print(f"ID: {record['id']}")
    print(f"생성일: {record['created_at']}")
    print(f"입력: {record['user_input'][:50]}...")
    print()

## 9. Socket.IO 서버 코드 (실제 사용)

아래 코드를 별도 파일(`streaming_server.py`)로 저장하여 실행할 수 있습니다.

In [ ]:
server_code = '''
"""
Deep Socket.IO Streaming Server

실행: python streaming_server.py
테스트: http://localhost:8000
"""
import os
import uuid
import asyncio
from datetime import datetime
from typing import TypedDict, List

import socketio
from aiohttp import web
from dotenv import load_dotenv

from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END

load_dotenv()

# ============================================================
# 설정
# ============================================================
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://localhost:11434")
MODEL_NAME = "gemma3:12b"

# Socket.IO 서버
sio = socketio.AsyncServer(async_mode="aiohttp", cors_allowed_origins="*")
app = web.Application()
sio.attach(app)

# LLM
llm = ChatOllama(model=MODEL_NAME, base_url=OLLAMA_URL, temperature=0.7)


# ============================================================
# Mock Database
# ============================================================
class MockDatabase:
    def __init__(self):
        self.records: List[dict] = []
    
    async def save(self, data: dict) -> dict:
        await asyncio.sleep(0.3)
        record = {
            "id": str(uuid.uuid4()),
            "created_at": datetime.now().isoformat(),
            **data
        }
        self.records.append(record)
        return record

mock_db = MockDatabase()


# ============================================================
# Graph State & Nodes
# ============================================================
class GraphState(TypedDict):
    user_input: str
    analysis: str
    response: str
    saved_record: dict


async def analyzer_node(state: GraphState) -> GraphState:
    prompt = f"""다음 입력을 분석해주세요 (의도, 키워드, 감정):
{state["user_input"]}"""
    response = await llm.ainvoke(prompt)
    return {"analysis": response.content}


async def generator_node(state: GraphState) -> GraphState:
    prompt = f"""분석 결과를 바탕으로 응답을 생성해주세요.
입력: {state["user_input"]}
분석: {state["analysis"]}"""
    response = await llm.ainvoke(prompt)
    return {"response": response.content}


async def saver_node(state: GraphState) -> GraphState:
    record = await mock_db.save({
        "user_input": state["user_input"],
        "analysis": state["analysis"],
        "response": state["response"],
    })
    return {"saved_record": record}


# 그래프 빌드
workflow = StateGraph(GraphState)
workflow.add_node("analyzer", analyzer_node)
workflow.add_node("generator", generator_node)
workflow.add_node("saver", saver_node)
workflow.add_edge(START, "analyzer")
workflow.add_edge("analyzer", "generator")
workflow.add_edge("generator", "saver")
workflow.add_edge("saver", END)
graph = workflow.compile()


# ============================================================
# Socket.IO Events
# ============================================================
@sio.event
async def connect(sid, environ):
    print(f"[연결] {sid}")
    await sio.emit("connected", {"sid": sid}, to=sid)


@sio.event
async def disconnect(sid):
    print(f"[연결 해제] {sid}")


@sio.event
async def chat(sid, data):
    """채팅 메시지 처리 - 전체 이벤트 스트리밍"""
    user_input = data.get("message", "")
    print(f"[{sid}] 입력: {user_input}")
    
    current_node = None
    
    try:
        async for event in graph.astream_events(
            {"user_input": user_input},
            version="v2"
        ):
            kind = event["event"]
            
            # 노드 시작
            if kind == "on_chain_start":
                node_name = event.get("name", "")
                if node_name in ["analyzer", "generator", "saver"]:
                    current_node = node_name
                    await sio.emit("node_start", {"node": node_name}, to=sid)
            
            # 노드 종료
            elif kind == "on_chain_end":
                node_name = event.get("name", "")
                if node_name in ["analyzer", "generator", "saver"]:
                    output = event.get("data", {}).get("output", {})
                    await sio.emit("node_end", {
                        "node": node_name,
                        "output": str(output)[:500]  # 출력 일부만
                    }, to=sid)
                    
                    # DB 저장 이벤트
                    if node_name == "saver" and "saved_record" in output:
                        await sio.emit("db_save", output["saved_record"], to=sid)
            
            # LLM 토큰 스트리밍
            elif kind == "on_chat_model_stream":
                chunk = event.get("data", {}).get("chunk")
                if chunk and chunk.content:
                    await sio.emit("token", {
                        "node": current_node,
                        "content": chunk.content
                    }, to=sid)
        
        # 완료
        await sio.emit("done", {"success": True}, to=sid)
    
    except Exception as e:
        await sio.emit("error", {"message": str(e)}, to=sid)


# ============================================================
# HTML 테스트 페이지
# ============================================================
HTML_PAGE = """
<!DOCTYPE html>
<html>
<head>
    <title>Deep Streaming Test</title>
    <script src="https://cdn.socket.io/4.7.2/socket.io.min.js"></script>
    <style>
        body { font-family: monospace; padding: 20px; background: #1a1a2e; color: #eee; }
        #output { background: #16213e; padding: 20px; border-radius: 8px; height: 400px; overflow-y: auto; }
        .node-start { color: #00ff88; }
        .node-end { color: #00ccff; }
        .token { color: #fff; }
        .db-save { color: #ffcc00; }
        .done { color: #ff88ff; font-weight: bold; }
        .error { color: #ff4444; }
        input { width: 70%; padding: 10px; font-size: 16px; }
        button { padding: 10px 20px; font-size: 16px; cursor: pointer; }
    </style>
</head>
<body>
    <h1>🚀 Deep Socket.IO Streaming</h1>
    <input type="text" id="input" placeholder="메시지를 입력하세요..." />
    <button onclick="sendMessage()">전송</button>
    <div id="output"></div>
    
    <script>
        const socket = io();
        const output = document.getElementById('output');
        
        function log(msg, cls = '') {
            output.innerHTML += `<div class="${cls}">${msg}</div>`;
            output.scrollTop = output.scrollHeight;
        }
        
        socket.on('connected', (data) => log(`✅ 연결됨: ${data.sid}`));
        
        socket.on('node_start', (data) => {
            log(`<br>🚀 [NODE START] ${data.node}`, 'node-start');
        });
        
        socket.on('node_end', (data) => {
            log(`<br>✅ [NODE END] ${data.node}`, 'node-end');
        });
        
        socket.on('token', (data) => {
            // 토큰은 줄바꿈 없이 이어붙임
            const lastDiv = output.lastElementChild;
            if (lastDiv && lastDiv.classList.contains('token')) {
                lastDiv.innerHTML += data.content;
            } else {
                log(data.content, 'token');
            }
            output.scrollTop = output.scrollHeight;
        });
        
        socket.on('db_save', (data) => {
            log(`<br>💾 [DB SAVE] ID: ${data.id}`, 'db-save');
        });
        
        socket.on('done', () => {
            log(`<br>🎉 [DONE] 완료!`, 'done');
        });
        
        socket.on('error', (data) => {
            log(`<br>❌ [ERROR] ${data.message}`, 'error');
        });
        
        function sendMessage() {
            const input = document.getElementById('input');
            const message = input.value.trim();
            if (message) {
                log(`<br>📤 [SEND] ${message}`, 'node-start');
                socket.emit('chat', { message });
                input.value = '';
            }
        }
        
        document.getElementById('input').addEventListener('keypress', (e) => {
            if (e.key === 'Enter') sendMessage();
        });
    </script>
</body>
</html>
"""

async def index(request):
    return web.Response(text=HTML_PAGE, content_type="text/html")

app.router.add_get("/", index)


# ============================================================
# 서버 실행
# ============================================================
if __name__ == "__main__":
    print(f"Ollama URL: {OLLAMA_URL}")
    print(f"Model: {MODEL_NAME}")
    print("\n🚀 서버 시작: http://localhost:8000")
    web.run_app(app, host="0.0.0.0", port=8000)
'''

# 파일로 저장
server_file_path = "streaming_server.py"
with open(server_file_path, "w") as f:
    f.write(server_code)

print(f"✅ 서버 코드가 '{server_file_path}'로 저장되었습니다.")
print("\n실행 방법:")
print(f"  python {server_file_path}")
print("\n테스트:")
print("  http://localhost:8000 접속")

## 10. 이벤트 흐름 요약

```
클라이언트                     서버                          LangGraph
    │                           │                               │
    │──── chat {message} ──────>│                               │
    │                           │──── astream_events() ────────>│
    │                           │                               │
    │<── node_start: analyzer ──│<── on_chain_start: analyzer ──│
    │<── token ─────────────────│<── on_chat_model_stream ──────│
    │<── token ─────────────────│<── on_chat_model_stream ──────│
    │<── node_end: analyzer ────│<── on_chain_end: analyzer ────│
    │                           │                               │
    │<── node_start: generator ─│<── on_chain_start: generator ─│
    │<── token ─────────────────│<── on_chat_model_stream ──────│
    │<── token ─────────────────│<── on_chat_model_stream ──────│
    │<── node_end: generator ───│<── on_chain_end: generator ───│
    │                           │                               │
    │<── node_start: saver ─────│<── on_chain_start: saver ─────│
    │<── node_end: saver ───────│<── on_chain_end: saver ───────│
    │<── db_save {record} ──────│                               │
    │                           │                               │
    │<── done ──────────────────│                               │
```

## 11. 요약

### Socket.IO 이벤트

| 이벤트 | 방향 | 데이터 |
|--------|------|--------|
| `chat` | Client → Server | `{message: string}` |
| `node_start` | Server → Client | `{node: string}` |
| `node_end` | Server → Client | `{node: string, output: string}` |
| `token` | Server → Client | `{node: string, content: string}` |
| `db_save` | Server → Client | `{id, created_at, ...}` |
| `done` | Server → Client | `{success: boolean}` |
| `error` | Server → Client | `{message: string}` |

### 핵심 코드

```python
async for event in graph.astream_events(input, version="v2"):
    kind = event["event"]
    
    if kind == "on_chain_start" and event["name"] in NODES:
        await sio.emit("node_start", {"node": event["name"]}, to=sid)
    
    elif kind == "on_chat_model_stream":
        await sio.emit("token", {"content": chunk.content}, to=sid)
    
    elif kind == "on_chain_end" and event["name"] in NODES:
        await sio.emit("node_end", {"node": event["name"]}, to=sid)
```